In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
--2019-04-10 01:56:04--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.222, 2001:67c:1560:8003::8004
Connecting to launchpad.net (launchpad.net)|91.189.89.222|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-10 01:56:04 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocamlfuse_0.7.0-0ubuntu1_

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
from sklearn.preprocessing import LabelBinarizer

In [0]:
def load_data(fold_path):
  data = load_files(fold_path)
  files = np.array(data['filenames'])
  targ = np.array(data['target'])
  one_hot = LabelBinarizer()
  ohd = one_hot.fit_transform(targ)
  return files,ohd

In [0]:
def ext_im(im_file):
  img = cv2.imread(im_file)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(256,256))
  img = img/255
  return img

In [0]:
def ext_im_gr(im_file):
  img = cv2.imread(im_file)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img,(256,256))
  img = np.expand_dims(img,axis=2)
  img = img/255
  return img

In [0]:
def tensor_4d(fil):
  lis = [ext_im(im) for im in tqdm(fil)]
  return np.stack(lis,axis=0)

In [0]:
def tensor_4d_gr(fil):
  lis = [ext_im_gr(im) for im in tqdm(fil)]
  return np.stack(lis,axis=0)

In [0]:
def get_filter(shape,na):
  with tf.variable_scope('unet'+na,reuse=tf.AUTO_REUSE):
    w = tf.get_variable(name=na,shape=shape,dtype='float32',initializer=tf.random_normal_initializer(dtype='float32',mean=0.0,stddev=1.0))
  return w

In [0]:
def dice_cof(inp_img,pred_img):
  inp = tf.reshape(inp_img,shape=[-1])
  pred = tf.reshape(pred_img,shape=[-1])
  smooth = 1.0
  intersection = tf.reduce_sum(inp*pred)
  score = (2.* intersection + smooth)/(tf.reduce_sum(inp) + tf.reduce_sum(pred)+smooth)
  return score
  

In [0]:
def dice_loss(inp,pred):
  loss = (- dice_cof(inp,pred))
  return loss

In [0]:
def attn_blk(gating_signal,input_signal,old_filters,new_filters,name):
  gat_1 = tf.nn.conv2d(gating_signal,filter=get_filter(shape=[1,1,old_filters,new_filters],na=name+'_f1'),stride=[1,1,1,1],padding='SAME',name='ag_'+ name)
  gm,gv = tf.nn.moments(gat_1,axes=[0,1,2])
  g1 = tf.nn.batch_normalization(gat_1,mean=gm,variance=gv,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='ag_bn_'+name)
  
  ins_1 = tf.nn.conv2d(input_signal,filter=get_filter(shape=[1,1,old_filters,new_filters],na=name+'_x1'),stride=[1,1,1,1],padding='SAME',name='ax_'+ name)
  im,iv = tf.nn.moments(ins_1,axes=[0,1,2])
  x1 = tf.nn.batch_normalization(ins_1,mean=im,variance=iv,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='ax_bn_'+name)
  
  comb_signal = tf.nn.relu(g1+x1)
  
  alpha_1 = tf.nn.conv2d(comb_signal,filter=get_filter(shape=[1,1,new_filters,1],na=name+'_alpha1'),stride=[1,1,1,1],padding='SAME',name='alpha_'+ name)
  am,av =  tf.nn.moments(alpha_1,axes=[0,1,2])
  alpha_2 =  tf.nn.batch_normalization(ins_1,mean=im,variance=iv,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='alpha_bn_'+name)
  alpha = tf.nn.sigmoid(alpha_2,name='alpha_final'+name)
  return alpha

In [0]:
def attn_unet(inp):
  lay_16_1 = tf.nn.conv2d(inp,filter=get_filter(shape=[3,3,3,16],na='w_1'),stride=[1,1,1,1],padding='SAME',name='conv_16_1')
  lay_16_2 = tf.nn.relu(lay_16_1,name='re_16_1')
  m1,v1 = tf.nn.moments(lay_16_2,axes=[0,1,2])
  lay_16_3 = tf.nn.batch_normalization(lay_16_2,mean=m1,variance=v1,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='bn_16')
  lay_16_4 = tf.nn.conv2d(lay_16_3,filter=get_filter(shape=[3,3,16,16],na='w_2'),stride=[1,1,1,1],padding='SAME',name='conv_16_2')
  lay_16_5 = tf.nn.relu(lay_16_4,name='re_16_2')
  lay_p1 = tf.nn.max_pool(lay_16_5)
  
  lay_32_1 = tf.nn.conv2d(lay_p1,filter=get_filter(shape=[3,3,16,32],na='w_3'),stride=[1,1,1,1],padding='SAME',name='conv_32_1')
  lay_32_2 = tf.nn.relu(lay_32_1,name='re_32_1')
  m2,v2 = tf.nn.moments(lay_32_2,axes=[0,1,2])
  lay_32_3 = tf.nn.batch_normalization(lay_32_2,mean=m2,variance=v2,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='bn_32')
  lay_32_4 = tf.nn.conv2d(lay_32_3,filter=get_filter(shape=[3,3,32,32],na='w_4'),stride=[1,1,1,1],padding='SAME',name='conv_32_2')
  lay_32_5 = tf.nn.relu(lay_32_4,name='re_32_2')
  lay_p2 = tf.nn.max_pool(lay_32_5)
  
  lay_64_1 = tf.nn.conv2d(lay_p2,filter=get_filter(shape=[3,3,32,64],na='w_5'),stride=[1,1,1,1],padding='SAME',name='conv_64_1')
  lay_64_2 = tf.nn.relu(lay_64_1,name='re_64_1')
  m3,v3 = tf.nn.moments(lay_64_2,axes=[0,1,2])
  lay_64_3 = tf.nn.batch_normalization(lay_64_2,mean=m3,variance=v3,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='bn_64')
  lay_64_4 = tf.nn.conv2d(lay_64_3,filter=get_filter(shape=[3,3,64,64],na='w_6'),stride=[1,1,1,1],padding='SAME',name='conv_64_2')
  lay_64_5 = tf.nn.relu(lay_64_4,name='re_64_2')
  lay_p3 = tf.nn.max_pool(lay_64_5)
  
  lay_128_1 = tf.nn.conv2d(lay_p3,filter=get_filter(shape=[3,3,64,128],na='w_7'),stride=[1,1,1,1],padding='SAME',name='conv_128_1')
  lay_128_2 = tf.nn.relu(lay_128_1,name='re_128_1')
  m4,v4 = tf.nn.moments(lay_128_2,axes=[0,1,2])
  lay_128_3 = tf.nn.batch_normalization(lay_128_2,mean=m4,variance=v4,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='bn_128')
  lay_128_4 = tf.nn.conv2d(lay_128_3,filter=get_filter(shape=[3,3,32,32],na='w_8'),stride=[1,1,1,1],padding='SAME',name='conv_128_2')
  lay_128_5 = tf.nn.relu(lay_128_4,name='re_128_2')
  lay_p3 = tf.nn.max_pool(lay_128_5)